### Book Recommendation with Keras

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, Embedding, Dot, Concatenate, Input
from keras.models import Model

#### Rating in the dataset

In [6]:
# rating_dataset

rating = pd.read_csv('Ratings.csv')

# books_dataset

books = pd.read_csv('Books.csv')

In [7]:
rating.head() 

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


The Id of the book, the userId and the rating given to the book by the user are specified in the column

In [8]:
books.head()

,Unnamed: 0,ISBN,book_id,Publication Year,Author,Title,AvgRating,Image-URL,Image-URL-S
0,0,0195153448,1,2008,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.34,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,0002005018,2,1997,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,4.44,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,0060973129,3,2005,Stephenie Meyer,"Twilight (Twilight, #1)",3.57,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,3,0374157065,4,1960,Harper Lee,To Kill a Mockingbird,4.25,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,4,0393045218,5,1925,F. Scott Fitzgerald,The Great Gatsby,3.89,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [9]:
# unique books and users
n_users = rating.user_id.nunique()
n_books = rating.book_id.nunique()

print("The number of users -", n_users)
print("The number of books -", n_books)

The number of users - 53424
The number of books - 10000


In [10]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981756 entries, 0 to 981755
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   book_id  981756 non-null  int64
 1   user_id  981756 non-null  int64
 2   rating   981756 non-null  int64
dtypes: int64(3)
memory usage: 22.5 MB


In [12]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        10000 non-null  int64  
 1   ISBN              10000 non-null  object 
 2   book_id           10000 non-null  int64  
 3   Publication Year  10000 non-null  int64  
 4   Author            10000 non-null  object 
 5   Title             10000 non-null  object 
 6   AvgRating         10000 non-null  float64
 7   Image-URL         10000 non-null  object 
 8   Image-URL-S       10000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 703.2+ KB


#### Splitting the dataset into train and test

In [13]:
train, test = train_test_split(rating, test_size=0.3, random_state=13)

#### Embeddings and Model Architecture

In [14]:
# create book embedding path

book_input = Input(shape=[1], name="Book-Input")
book_embedding= Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding) #converting into matrix

In [15]:
# create user embedding path

user_input = Input(shape=[1], name="User-Input")
user_embedding= Embedding(n_books+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [18]:
# concatenate features

conc = Concatenate()([book_vec, user_vec])

In [21]:
# add fully connected layers 

fc1 = Dense(128, activation="relu")(conc)
fc2 = Dense(32, activation="relu")(fc1)
output= Dense(1)(fc2)

In [22]:
# create a model and compile it
model = Model([user_input, book_input], output)
model.compile("adam", "mean_squared_error")


#### Train the model

In [23]:
history= model.fit(x=[train.user_id, train.book_id], y=train.rating, epochs=10, verbose=1)

Epoch 1/10
21476/21476 [==============================] - 130s 6ms/step - loss: 0.9338
Epoch 2/10
21476/21476 [==============================] - 127s 6ms/step - loss: 0.8699
Epoch 3/10
21476/21476 [==============================] - 129s 6ms/step - loss: 0.8588
Epoch 4/10
21476/21476 [==============================] - 128s 6ms/step - loss: 0.8495
Epoch 5/10
21476/21476 [==============================] - 127s 6ms/step - loss: 0.8419
Epoch 6/10
21476/21476 [==============================] - 128s 6ms/step - loss: 0.8352
Epoch 7/10
21476/21476 [==============================] - 128s 6ms/step - loss: 0.8287
Epoch 8/10
21476/21476 [==============================] - 127s 6ms/step - loss: 0.8229
Epoch 9/10
21476/21476 [==============================] - 127s 6ms/step - loss: 0.8177
Epoch 10/10
21476/21476 [==============================] - 127s 6ms/step - loss: 0.8134


#### Evaluation and prediction

In [24]:
# Evaluation
model.evaluate([test.user_id, test.book_id], y=test.rating)

9204/9204 [==============================] - 26s 3ms/step - loss: 0.8883


0.8883085250854492

In [25]:
# Predictions
predictions = model.predict([test.user_id.head(10), test.book_id.head(10)])

for i in range(0,10):
    print("predicted Rating -", predictions[i], "Actual rating -",test.rating.iloc[i])

1/1 [==============================] - 0s 126ms/step
predicted Rating - [3.6055088] Actual rating - 4
predicted Rating - [3.6102397] Actual rating - 4
predicted Rating - [3.7792113] Actual rating - 2
predicted Rating - [4.0546637] Actual rating - 5
predicted Rating - [3.628289] Actual rating - 2
predicted Rating - [3.406718] Actual rating - 5
predicted Rating - [3.721266] Actual rating - 4
predicted Rating - [3.5898309] Actual rating - 3
predicted Rating - [4.587221] Actual rating - 5
predicted Rating - [2.6353424] Actual rating - 4


#### Making recommendation

In [26]:
#create dataset for making recommendations for the picked user

picked_userid = 150
book_data = np.array(list(set(test.book_id)))
book_data

array([    1,     2,     3, ...,  9998,  9999, 10000])

In [27]:
user=np.array([picked_userid for i in range(len(book_data))])
user

array([150, 150, 150, ..., 150, 150, 150])

In [30]:
predict_1= model.predict([user, book_data])

predictions= np.array([a[0] for a in predict_1])

recommended_book_ids=(-predictions).argsort()[:5]

recommended_book_ids

#print predicted score

print(predictions[recommended_book_ids])

313/313 [==============================] - 1s 2ms/step
[5.0649934 5.036704  5.0277853 4.9777    4.9375057]


In [31]:
# recommended books for picked user 
books[books["book_id"].isin(recommended_book_ids)]

,Unnamed: 0,ISBN,book_id,Publication Year,Author,Title,AvgRating,Image-URL,Image-URL-S
1306,1306,1573225789,1307,2011,Caitlin Moran,How to Be a Woman,3.73,https://images.gr-assets.com/books/1405909800m...,https://images.gr-assets.com/books/1405909800s...
3751,3751,089190672X,3752,1924,Agatha Christie,"Poirot Investiga (Hércules Poirot, #3)",4.07,https://images.gr-assets.com/books/1359475912m...,https://images.gr-assets.com/books/1359475912s...
4105,4105,0380713330,4106,1864,"Fyodor Dostoyevsky, Christian Redl, Philip Dos...",Notes from Underground,4.15,https://images.gr-assets.com/books/1327909683m...,https://images.gr-assets.com/books/1327909683s...
4776,4776,0060920076,4777,1930,William Faulkner,A Rose for Emily and Other Stories,4.06,https://images.gr-assets.com/books/1487468453m...,https://images.gr-assets.com/books/1487468453s...
8976,8976,039914840X,8977,2012,A.J. Jacobs,Drop Dead Healthy: One Man's Humble Quest for ...,3.75,https://images.gr-assets.com/books/1344398815m...,https://images.gr-assets.com/books/1344398815s...
